In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

fn = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        fn.append(os.path.join(dirname, filename))
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/drug-classification/drug200.csv


In [2]:
import pandas as pd

# Reading the CSV file
df = pd.read_csv(fn[0])

# Displaying the first few rows of the DataFrame
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# # Sample DataFrame
# data = {
#     'Age': [23, 45, 56, 67],
#     'Sex': ['F', 'M', 'F', 'M'],
#     'BP': ['HIGH', 'LOW', 'NORMAL', 'HIGH'],
#     'Cholesterol': ['NORMAL', 'HIGH', 'NORMAL', 'HIGH'],
#     'Na_to_K': [15.0, 13.5, 14.2, 16.0],
#     'Drug': ['drugY', 'drugC', 'drugX', 'drugA']
# }

# df = pd.DataFrame(data)

# Columns to label encode
columns_to_encode = ['Sex', 'BP', 'Cholesterol', 'Drug']

# Initialize the LabelEncoder
le = LabelEncoder()

# Apply label encoding to each column
for col in columns_to_encode:
    df[col] = le.fit_transform(df[col])

# Display the new DataFrame
df

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,0,0,0,25.355,0
1,47,1,1,0,13.093,3
2,47,1,1,0,10.114,3
3,28,0,2,0,7.798,4
4,61,0,1,0,18.043,0
...,...,...,...,...,...,...
195,56,0,1,0,11.567,3
196,16,1,1,0,12.006,3
197,52,1,2,0,9.894,4
198,23,1,2,1,14.020,4


In [4]:
from sklearn.model_selection import train_test_split

# Features and target
X = df.drop('Drug', axis=1)
y = df['Drug']

# Split into train and temp (test + validation)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)

# Split temp into test and validation
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Display the shapes of the resulting DataFrames
print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Testing set: {X_test.shape}, {y_test.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")

Training set: (120, 5), (120,)
Testing set: (40, 5), (40,)
Validation set: (40, 5), (40,)


In [5]:
# Assuming y_train, y_test, and y_val are already defined

# Unique counts in y_train
train_counts = y_train.value_counts()
print("Unique counts in y_train:")
print(train_counts)

# Unique counts in y_test
test_counts = y_test.value_counts()
print("\nUnique counts in y_test:")
print(test_counts)

# Unique counts in y_val
val_counts = y_val.value_counts()
print("\nUnique counts in y_val:")
print(val_counts)


Unique counts in y_train:
Drug
0    55
4    32
1    14
3    10
2     9
Name: count, dtype: int64

Unique counts in y_test:
Drug
0    18
4    11
1     4
2     4
3     3
Name: count, dtype: int64

Unique counts in y_val:
Drug
0    18
4    11
1     5
2     3
3     3
Name: count, dtype: int64


In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Assuming x_train, x_test, y_train, y_test, x_val, y_val are already defined

# Define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Decision Tree classifier
dt = DecisionTreeClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=dt,
                           param_grid=param_grid, 
                           cv=5, scoring='accuracy', n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Test the best model
y_pred_test = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Evaluate on validation data
y_pred_val = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print(f"Validation Accuracy: {val_accuracy:.4f}")

Test Accuracy: 0.9750
Validation Accuracy: 1.0000


In [7]:
df

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,0,0,0,25.355,0
1,47,1,1,0,13.093,3
2,47,1,1,0,10.114,3
3,28,0,2,0,7.798,4
4,61,0,1,0,18.043,0
...,...,...,...,...,...,...
195,56,0,1,0,11.567,3
196,16,1,1,0,12.006,3
197,52,1,2,0,9.894,4
198,23,1,2,1,14.020,4


In [8]:
import pandas as pd
from imblearn.over_sampling import SMOTE

# Assuming df is your DataFrame and 'target' is the column you want to balance
X = df.drop('Drug', axis=1)
y = df['Drug']

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Create a new DataFrame with the resampled data
df_resampled = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=['Drug'])], axis=1)

df_resampled['Drug'].value_counts()

Drug
0    91
3    91
4    91
1    91
2    91
Name: count, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

# Features and target
X = df_resampled.drop('Drug', axis=1)
y = df_resampled['Drug']

# Split into train and temp (test + validation)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)

# Split temp into test and validation
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Display the shapes of the resulting DataFrames
print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Testing set: {X_test.shape}, {y_test.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")

Training set: (273, 5), (273,)
Testing set: (91, 5), (91,)
Validation set: (91, 5), (91,)


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Assuming x_train, x_test, y_train, y_test, x_val, y_val are already defined

# Define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Decision Tree classifier
dt = DecisionTreeClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=dt,
                           param_grid=param_grid, 
                           cv=5, scoring='accuracy', n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Test the best model
y_pred_test = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Evaluate on validation data
y_pred_val = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print(f"Validation Accuracy: {val_accuracy:.4f}")

Test Accuracy: 0.9890
Validation Accuracy: 0.9890
